In [1]:
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt

Setting some default values

In [2]:
img_lib = "x"
lab_lib = "y"

num_training_images = 300
training_start_index = 1500
training_end_index = training_start_index+num_training_images

test_starting_index = 2800
num_test_images = 50
test_end_index = test_starting_index+num_test_images

img_paths = glob.glob(img_lib+"/*.png")
lab_paths = glob.glob(lab_lib+"/*.png")

img_num = len(img_paths)
lab_num = len(lab_paths)
message = "The number of images found is {0:4d} and the number of labels is {1:4d}".format(img_num, lab_num)
print(message)

#ensuring that they are in the same order:

img_paths.sort()
lab_paths.sort()

#creating the dictionary with the selected slices of paths

training_img_paths = {"x": img_paths[training_start_index:training_end_index], "y": lab_paths[training_start_index:training_end_index]}
test_img_paths = {"x": img_paths[test_starting_index:test_end_index], "y": lab_paths[test_starting_index:test_end_index]}


The number of images found is 3000 and the number of labels is 3000


In [3]:
from loading_helpers import *

In [4]:
import time
print("Loading %d images has started."%(num_training_images))
start_time = time.time()
ready_im, ready_lab = load_imgs_parallel(training_img_paths, num_slices=20)
end_time= time.time()
print("Loading the images has taken %d seconds."%(end_time-start_time))

Loading 300 images has started.
feed array's shape:(300, 2)
   1 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
   2 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
   3 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
   4 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
   5 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
   6 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
   7 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
   8 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
   9 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
  10 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
  11 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
  12 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
  13 slice is being processed out of   20 slice.
(2, 60, 640, 640, 3)
  14 slice is being processed 

Tesing whether the images and labels match...

In [5]:
#visual_test_n_loaded_imgs(5, ready_im, ready_lab)

In [6]:
from get_unet import *
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras import losses
input_img = Input((640,640,1))
model = get_unet(input_img, n_filters = 16, dropout = 0.5, batchnorm=True, passed_activation = "sigmoid")

W0722 22:23:18.254234 139829090731392 deprecation_wrapper.py:119] From /home/contact_gergo_endresz/j/get_unet.py:2: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.

W0722 22:23:18.255571 139829090731392 deprecation_wrapper.py:119] From /home/contact_gergo_endresz/j/get_unet.py:2: The name tf.keras.layers.CuDNNLSTM is deprecated. Please use tf.compat.v1.keras.layers.CuDNNLSTM instead.



hello


W0722 22:23:19.067826 139829090731392 deprecation.py:506] From /home/contact_gergo_endresz/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
selected_loss = losses.binary_crossentropy

model.compile("adam", loss=selected_loss, metrics=["accuracy", losses.mean_squared_error, losses.logcosh, losses.kullback_leibler_divergence,  losses.binary_crossentropy, losses.mean_squared_logarithmic_error])

#model.load_weights("./ckpts/2d_sigm_norm.hdf5")

checkpoint = tf.keras.callbacks.ModelCheckpoint("./ckpts/2d_sigm_norm_br1.hdf5", save_best=1, monitor="val_loss", mode="auto")
tboard = tf.keras.callbacks.TensorBoard("./ckpts/tb3/", histogram_freq=1, batch_size=20, write_graph = False, write_grads = False, write_images = True, update_freq = "epoch")
callbacks = [checkpoint, tboard]

datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2, brightness_range=[0.3, 2])#,
                               #rotation_range=15,
                               #width_shift_range=0.1,
                               #height_shift_range=0.1,
                               #shear_range=0.01,
                               #zoom_range=[0.5, 1.25],
                               #horizontal_flip=True,
                               #vertical_flip=True,
                               #fill_mode='reflect',
                               #rotation_range=360, fill_mode = "reflect", horizontal_flip=1, vertical_flip=1,)
#datagen = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_std_normalization= 1, rotation_range=360, fill_mode = "reflect", horizontal_flip=1, vertical_flip=1, validation_split=0.2)
#you have to use datagen.fit if u want to use featurewise_std_normalization so that the generator learns about the features

W0722 22:23:20.109578 139829090731392 deprecation.py:323] From /home/contact_gergo_endresz/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model.fit_generator(datagen.flow(x=ready_im, y=ready_lab , batch_size = 20, shuffle=True), steps_per_epoch = 10, epochs = 100,verbose = True, callbacks = callbacks, use_multiprocessing = False)

Epoch 1/100


In [ ]:
pred_im, pred_lab = load_imgs_parallel(test_img_paths, num_slices=10)

In [ ]:
preds = model.predict(x=ready_im, batch_size = 10)

In [ ]:
print(preds.shape)

In [ ]:
visual_test_n_preds(5, ready_lab, preds, 0)